Dependencies

In [24]:
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    DataCollatorForLanguageModeling, 
    Trainer, 
    TrainingArguments
)

from datasets import load_dataset

In [25]:
dataset = load_dataset('hakurei/open-instruct-v1', split='train')
dataset.to_pandas().sample(20)

Found cached dataset json (C:/Users/cristian.sanchezp/.cache/huggingface/datasets/hakurei___json/hakurei--open-instruct-v1-00713eb9aefc6002/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


,output,instruction,input
56572,"狗, 猫","Vocabulary problem (inputs: words in english, ...",
355613,"(3.0, 2.0, 4.0, 1)",Make a function that takes an array of numbers...,
392748,Object-oriented programming (OOP) is a program...,Describe what is meant by object-oriented prog...,
344104,Driver,Find out who is responsible for the following ...,
296291,A potential challenge for the GPT model when t...,Give an example of a potential challenge the G...,
197926,int index = 4,"What is the first index of the substring ""mod""...",
55520,def average_temperature(city):\n temp = 0.0...,I want to know the average temperature of a gi...,
498796,A diplomatic answer to handle a disagreement a...,Give me a diplomatic answer to handle a disagr...,
334732,The future of machine learning looks bright. W...,Write an article for a magazine about a topic ...,
138959,Opportunity cost refers to the potential benef...,Define the term 'opportunity cost' and provide...,


In [26]:
def preprocess(example):
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"
    return example

In [27]:
dataset = dataset.map(preprocess, remove_columns=['instruction','input','output'])

Loading cached processed dataset at C:\Users\cristian.sanchezp\.cache\huggingface\datasets\hakurei___json\hakurei--open-instruct-v1-00713eb9aefc6002\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-cbda1d7d0dbdd8ca.arrow


In [28]:
dataset.to_pandas().sample(10)

,prompt
137097,Describe how to set up a basic home security s...
250179,What is the recipe to make non-spicy Awadhi ch...
3075,Rewrite the following sentence using correct c...
288131,Describe the characteristics of a cat in 1 sen...
150781,Come up with a solution for the following math...
479249,How do I maximize the yield when growing veget...
310009,Provide the main reasons why a knife is a more...
299123,Complete the following sentence using the give...
95115,Tell me about a time when you went out of your...
1603,Generate a list of 5 items we are most likely ...


In [29]:
dataset = dataset.shuffle(seed=42).select(range(100000)).train_test_split(test_size=0.1)

Loading cached shuffled indices for dataset at C:\Users\cristian.sanchezp\.cache\huggingface\datasets\hakurei___json\hakurei--open-instruct-v1-00713eb9aefc6002\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-2bc5d54f439219b6.arrow


In [30]:
train_dataset = dataset['train']
test_dataset = dataset['test']

Model Selection

In [31]:
MODEL_NAME = "microsoft/DialoGPT-medium"

In [32]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [33]:
def tokenize_dataset(dataset):
    return dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])

In [34]:
tokenizer.pad_token = tokenizer.eos_token

In [35]:
train_dataset = tokenize_dataset(train_dataset)
test_dataset = tokenize_dataset(test_dataset)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [36]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 90000
})

In [ ]:
test_

In [13]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

The trainer will use this to process the input and create appropiate batches for training, as we use the generative model, the mlm will be set to false

In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
training_args = TrainingArguments(
    output_dir='../../data/interim/',
    num_train_epochs=1, #To keep things fast
    per_device_eval_batch_size=4,
    per_device_train_batch_size=8,
)

In [18]:
#Will do all the heavy lifting
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=test_dataset, 
    data_collator=data_collator,
)

In [19]:
trainer.train()
trainer.save_model()

  0%|          | 0/5625 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: [enforce fail at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 408488896 bytes.